In [7]:
from datasets import load_dataset
import torch
from tqdm.auto import tqdm
from utils.zero_shot_baselines import return_lm_baseline_zero_shot, nli_based_zero_shot

dataset_l = load_dataset("pubmed_qa", "pqa_labeled")

Reusing dataset pubmed_qa (/home/debo/.cache/huggingface/datasets/pubmed_qa/pqa_labeled/1.0.0/2e65addecca4197502cd10ab8ef1919a47c28672f62d7abac7cc9afdcf24fb2d)


In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

model = AutoModelForMaskedLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

for k in range(10):
    sample = dataset_l['train'][k]
    premise = ','.join(sample['context']['contexts'])  
    sequence = f"{premise} {sample['question']}? Answer:  {tokenizer.mask_token}"
    res = return_lm_baseline_zero_shot(model, tokenizer, sequence)
    print(res)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['?', '.', "'", ':', ')', ']', '(', '!', '-', '•', '/', '*', '[', '·', '1']
['?', '.', '!', ':', "'", ']', ')', '(', '-', ';', '•', '[', '*', '>', 'a']
['.', '?', "'", '!', ':', ')', ']', '(', '`', '-', '/', '[', '¿', '1', ';']
['?', '.', "'", '!', ':', ')', ']', '(', '}', '[', '¿', '-', '#', '*', '/']
['.', '?', "'", ':', ')', '!', ']', '(', ',', '*', '[', '¿', '-', '}', ';']
['?', '.', "'", '!', ':', ']', ')', '(', '}', '*', '/', '[', '•', '>', '-']
['?', '.', "'", '!', ':', ']', ')', '(', '*', '}', '-', 'to', '>', '[', '#']
['?', '.', "'", ':', ')', ']', '!', '(', '-', '[', '¿', '`', '·', '*', '/']
['?', '.', "'", '!', ']', ')', ':', '(', '}', '¿', '-', '[', '`', 'do', ',']
['?', '.', "'", ':', ')', '!', ']', '(', '/', '[', '*', '-', '}', '`', 'to']


In [9]:
from transformers import BartForSequenceClassification, BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')


In [10]:
ctr = 0
num_items=100
for k in tqdm(range(num_items)):
    sample = dataset_l['train'][k]
    # pose sequence as a NLI premise and label answer as a hypothesis
    premise = ','.join(sample['context']['contexts']) + sample['question']
    # premise = f"Given the context {premise}"
    hypothesis = sample['long_answer']
    res = nli_based_zero_shot(model, tokenizer, premise, hypothesis)
    out = 'yes' if res>0.5 else 'no'
    ctr+= out==sample['final_decision']
print(f"Accuracy : {ctr/num_items*100}")

  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy : 59.0
